In [108]:
from langchain_community.document_loaders import WebBaseLoader, PyPDFDirectoryLoader, PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable

from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate

from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.chains import retrieval_qa

from langchain.llms import Ollama
from langchain_groq import ChatGroq

from langchain_community.tools.asknews import AskNewsSearch

from dotenv import load_dotenv
import os

In [91]:
str = input("Enter link: ")
loader = WebBaseLoader(str)

In [99]:
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
final_docs = text_splitter.split_documents(documents)
title = final_docs[0].metadata['title']
description = final_docs[0].metadata['description']

In [107]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectors = FAISS.from_documents(final_docs, embeddings)

In [93]:
load_dotenv()
groq_api_key = os.environ['GROQ_API_KEY']
llm = ChatGroq(api_key=groq_api_key, model="gemma2-9b-it", temperature=0.4)

In [94]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a neutralizer assistant that converts news headline and description into a neutral, bias-free search query suitable for finding balanced articles on the topic. Focus only on the core subject matter and important details."),
    ("human", "Headline {headline}\nDescription: {description}"),
    ("system", "Neutral Query: ")
])

In [95]:
chain = prompt | llm
response = chain.invoke({"headline" : title, "description" : description})

In [96]:
response.content

'Indian Parliament debate on Operation Sindoor and political criticism \n'

In [ ]:
fair_prompt = ChatPromptTemplate.from_messages([
    ("system", "")
])

In [ ]:
retriever = vectors.as_retriever()
asknews_tool = AskNewsSearch()
tools = [asknews_tool]
agent = create_tool_calling_agent(llm, tools, fair_prompt)